cold-user에게 정보를 받는 법 고민해보기

In [64]:
import sys
import os

# 주피터 노트북 환경에서 __file__이 없으므로, 현재 워킹 디렉토리 기준으로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import random
from data_scraping.common.data_loader import load_ratings_data

df_ratings = load_ratings_data()
lot_rated_movie_ids = set(df_ratings.value_counts(subset='movie_id').sort_values(ascending=False).values[:100])

# 여기 있는 영화를 보셨나요?
selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))
lot_rated_movie_ids = lot_rated_movie_ids - selected_movie_ids

# 버튼 다시 누르면
selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))

/var/folders/bw/_nzx77m90994jql91xkcq2kc0000gn/T/ipykernel_87052/3231592725.py:16: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))
/var/folders/bw/_nzx77m90994jql91xkcq2kc0000gn/T/ipykernel_87052/3231592725.py:20: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))


# 수집한 거 불러오기

In [65]:
from modeling.utils.data_integration import DataIntegrator

# Firebase 초기화
from user_system.firebase_config import setup_firebase_config
import streamlit as st

# Firebase 설정
firebase_available = setup_firebase_config()
print(f"Firebase 초기화 상태: {firebase_available}")

Firebase 초기화 상태: True


In [66]:
# Firebase 초기화 확인
if firebase_available:
    print("✅ Firebase가 성공적으로 초기화되었습니다.")
    
    # DataIntegrator 초기화
    integrator = DataIntegrator()
    
    # Firebase 데이터 로드 테스트
    print("Firebase 데이터를 로드하는 중...")
    firebase_data = integrator.load_firebase_data()
    print(f"Firebase 데이터: {len(firebase_data)}개 평점")
    
    if not firebase_data.empty:
        print("Firebase 데이터 샘플:")
        print(firebase_data.head())
    else:
        print("⚠️ Firebase에서 데이터를 찾을 수 없습니다.")
else:
    print("❌ Firebase 초기화에 실패했습니다.")
    print("Firebase 설정 파일을 확인해주세요.")


✅ Firebase가 성공적으로 초기화되었습니다.
Firebase 데이터를 로드하는 중...
Firebase 데이터: 12개 평점
Firebase 데이터 샘플:
                        user_id movie_id  rating
0  SOMV3U3OEWT7LG9t2VMeNXKQchd2  m5xMKrW     1.0
1  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mJ5x8x5     5.0
2  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mJ5xvEO     3.5
3  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mJOV1RW     4.0
4  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mOVgp1d     5.0


In [67]:
# 데이터 통합 테스트
if firebase_available:
    print("=== 데이터 통합 테스트 ===")
    
    # 1. 기존 데이터 로드
    print("1. 기존 데이터를 로드하는 중...")
    original_data = integrator.load_original_data()
    print(f"기존 데이터: {len(original_data)}개 평점")
    
    # 2. Firebase 데이터 로드
    print("2. Firebase 데이터를 로드하는 중...")
    firebase_data = integrator.load_firebase_data()
    print(f"Firebase 데이터: {len(firebase_data)}개 평점")
    
    # 3. 데이터 통합
    if not original_data.empty or not firebase_data.empty:
        print("3. 데이터를 통합하는 중...")
        integrated_data = integrator.integrate_data(original_data, firebase_data)
        print(f"통합된 데이터: {len(integrated_data)}개 평점")
        
        if not integrated_data.empty:
            # 통계 정보
            stats = integrator.get_data_statistics(integrated_data)
            print("\n=== 통합된 데이터 통계 ===")
            print(f"총 평점 수: {stats['total_ratings']:,}")
            print(f"고유 사용자: {stats['unique_users']:,}")
            print(f"고유 영화: {stats['unique_movies']:,}")
            print(f"평균 평점: {stats['avg_rating']:.2f}")
            
            # 데이터 샘플
            print("\n=== 통합된 데이터 샘플 ===")
            print(integrated_data.head())
        else:
            print("⚠️ 통합된 데이터가 비어있습니다.")
    else:
        print("⚠️ 통합할 데이터가 없습니다.")
else:
    print("Firebase가 초기화되지 않아 데이터 통합을 진행할 수 없습니다.")


=== 데이터 통합 테스트 ===
1. 기존 데이터를 로드하는 중...
기존 데이터: 3361101개 평점
2. Firebase 데이터를 로드하는 중...
Firebase 데이터: 12개 평점
3. 데이터를 통합하는 중...
통합된 데이터: 3361113개 평점

=== 통합된 데이터 통계 ===
총 평점 수: 3,361,113
고유 사용자: 4,458
고유 영화: 49,856
평균 평점: 3.32

=== 통합된 데이터 샘플 ===
         user_id movie_id   movie_title  rating
0  ZBm5R0Mj7qd46  m5XKVDL         양자물리학     3.0
1  ZBm5R0Mj7qd46  m5Nn2GE       영화로운 작음     3.5
2  ZBm5R0Mj7qd46  mWvq0vZ           내 방     2.5
3  ZBm5R0Mj7qd46  mdKpyBk      성적표의 김민영     3.0
4  ZBm5R0Mj7qd46  mOopbwm  킹덤4: 대장군의 귀환     3.0


In [ ]:
from data_scraping.common.data_loader import load_movie_data, load_ratings_data

df_movies = load_movie_data()

In [69]:
integrated_data[integrated_data['user_id'] == 'SOMV3U3OEWT7LG9t2VMeNXKQchd2']

,user_id,movie_id,movie_title,rating
3361101,SOMV3U3OEWT7LG9t2VMeNXKQchd2,m5xMKrW,NaN,1.0
3361102,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJ5x8x5,NaN,5.0
3361103,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJ5xvEO,NaN,3.5
3361104,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJOV1RW,NaN,4.0
3361105,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mOVgp1d,NaN,5.0
3361106,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mObGGj7,NaN,4.0
3361107,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWpkJw5,NaN,4.5
3361108,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWqJnGr,NaN,4.0
3361109,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWw8zEO,NaN,1.0
3361110,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mY5QnJW,NaN,4.0


In [71]:
from modeling.utils.data import filter_by_min_counts

filtered = filter_by_min_counts(integrated_data, min_user_ratings=10)


[필터링 전] 사용자 수: 4,458명, 영화 수: 49,856개
[필터 적용] min_user_ratings: 10, min_movie_ratings: 10
조건 통과 사용자 수: 4,417명, 조건 통과 영화 수: 15,770개
[필터링 후] 사용자 수: 4,417명, 영화 수: 15,770개
필터링된 평점 수: 3,271,274개 (제거: 89,839개)


In [79]:
filtered[filtered['user_id'] == 'SOMV3U3OEWT7LG9t2VMeNXKQchd2']

,user_id,movie_id,movie_title,rating
3361101,SOMV3U3OEWT7LG9t2VMeNXKQchd2,m5xMKrW,NaN,1.0
3361102,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJ5x8x5,NaN,5.0
3361103,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJ5xvEO,NaN,3.5
3361104,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJOV1RW,NaN,4.0
3361105,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mOVgp1d,NaN,5.0
3361106,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mObGGj7,NaN,4.0
3361107,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWpkJw5,NaN,4.5
3361108,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWqJnGr,NaN,4.0
3361109,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWw8zEO,NaN,1.0
3361110,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mY5QnJW,NaN,4.0


In [52]:
# 'SOMV3U3OEWT7LG9t2VMeNXKQchd2' 사용자의 평점 개수 추출
user_id = 'SOMV3U3OEWT7LG9t2VMeNXKQchd2'
user_rating_count = integrated_data[integrated_data['user_id'] == user_id].shape[0]
print(f"{user_id} 평점 개수:", user_rating_count)

SOMV3U3OEWT7LG9t2VMeNXKQchd2 평점 개수: 12


In [74]:
from models.svd import SVDRecommenderPipeline, ModelConfig
config = ModelConfig.from_yaml()

pipeline = SVDRecommenderPipeline(config)
model_path = '/Users/user/Desktop/movie_recommendation/modeling/models/pkls/trained_svd_pipeline.pkl'
loaded_pipeline = SVDRecommenderPipeline.load_model(str(model_path))

In [78]:
loaded_pipeline.config

ModelConfig(n_factors=15, n_epochs=10, lr_all=0.005, reg_all=0.02, random_state=42, verbose=True, test_size=0.2, min_user_ratings=10, min_movie_ratings=10, rating_scale=(0.5, 5.0), use_integrated_data=True)

In [75]:
loaded_pipeline.recommend_for_user(user_id='SOMV3U3OEWT7LG9t2VMeNXKQchd2', df_movies=df_movies)

ValueError: 사용자 ID 'SOMV3U3OEWT7LG9t2VMeNXKQchd2'를 찾을 수 없습니다.

In [2]:
firebase_data

,user_id,movie_id,rating
0,SOMV3U3OEWT7LG9t2VMeNXKQchd2,m5xMKrW,1.0
1,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJ5x8x5,5.0
2,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJ5xvEO,3.5
3,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJOV1RW,4.0
4,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mOVgp1d,5.0
5,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mObGGj7,4.0
6,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWpkJw5,4.5
7,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWqJnGr,4.0
8,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWw8zEO,1.0
9,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mY5QnJW,4.0


In [4]:
import pandas as pd

pd.concat([firebase_data, filtered_data])

,user_id,movie_id,rating,movie_title
0,SOMV3U3OEWT7LG9t2VMeNXKQchd2,m5xMKrW,1.0,NaN
1,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJ5x8x5,5.0,NaN
2,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJ5xvEO,3.5,NaN
3,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJOV1RW,4.0,NaN
4,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mOVgp1d,5.0,NaN
...,...,...,...,...
3361579,ld0q0rrGR56Xn,mmOb3kd,5.0,해리포터와 마법사의 돌
3361580,ld0q0rrGR56Xn,mWyQz6W,4.0,엣지 오브 투모로우
3361581,ld0q0rrGR56Xn,mAd7zBO,4.0,제5원소
3361582,ld0q0rrGR56Xn,mM5rmEd,4.5,해리포터와 불의 잔


In [1]:
import sys
import os

# 주피터 노트북 환경에서 __file__이 없으므로, 현재 워킹 디렉토리 기준으로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

"""
SVD 파이프라인 테스트 스크립트
"""
import logging
from pathlib import Path
from modeling.models.svd import SVDRecommenderPipeline, ModelConfig

# Firebase 초기화
from user_system.firebase_config import setup_firebase_config
from user_system.firebase_firestore import FirestoreManager

from data_scraping.common.data_loader import load_ratings_data
from modeling.utils.data import filter_by_min_counts

# Logger 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


logger.info("🚀 SVD 추천 시스템 파이프라인 테스트")

# YAML 파일에서 모델 설정 로드
config = ModelConfig.from_yaml()

# 데이터 로드 (config에서 통합 데이터 사용 여부 확인)
if getattr(config, 'use_integrated_data', True): # 사용할 떄 True
    logger.info("📊 Firebase 통합 데이터를 사용합니다...")
    
    # Firebase 초기화
    logger.info("🔥 Firebase를 초기화하는 중...")
    firebase_available = setup_firebase_config()
    if not firebase_available:
        logger.error("❌ Firebase 초기화에 실패했습니다. 기존 데이터를 사용합니다.")
        logger.info("📊 기존 데이터를 사용합니다...")
    else:
        logger.info("✅ Firebase 초기화 완료!")
        firestore_manager = FirestoreManager()
        logger.info("Firebase에서 사용자 상호작용 데이터를 로드하는 중...")
        firebase_data = firestore_manager.get_all_user_ratings()
        logger.info(f"  - Firebase 데이터: {len(firebase_data):,}개 평점")

        logger.info("📥 기존 데이터를 로드하는 중...")
        df_ratings = load_ratings_data()
        logger.info(f"  - 기존 데이터: {len(df_ratings):,}개 평점")
        
        logger.info("🔍 데이터를 필터링하는 중...")
        logger.info(f"  - 필터링 조건: 사용자당 최소 {config.min_user_ratings}개, 영화당 최소 {config.min_movie_ratings}개")            
        filtered_data = filter_by_min_counts(df_ratings, min_movie_ratings=config.min_movie_ratings, min_user_ratings=config.min_user_ratings)
        logger.info(f"  - 필터링된 데이터: {len(filtered_data):,}개 평점")
                    

        logger.info("✅ 통합된 데이터로 설정이 업데이트되었습니다.")
else:
    logger.info("📊 기존 데이터를 사용합니다...")

# 파이프라인 생성 및 전체 실행
pipeline = SVDRecommenderPipeline(config)
pipeline.run_full_pipeline(filtered_data=filtered_data, firebase_data=firebase_data)

2025-10-26 00:40:24.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 00:40:24.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 00:40:24.873 
  command:

    streamlit run /Users/user/miniconda3/envs/dev/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-10-26 00:40:24.873 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 00:40:24.874 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 00:40:24.874 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new ca


[필터링 전] 사용자 수: 4,457명, 영화 수: 49,856개
[필터 적용] min_user_ratings: 10, min_movie_ratings: 10
조건 통과 사용자 수: 4,416명, 조건 통과 영화 수: 15,770개


2025-10-26 00:40:28,891 - __main__ - INFO -   - 필터링된 데이터: 3,271,262개 평점
2025-10-26 00:40:28,891 - __main__ - INFO - ✅ 통합된 데이터로 설정이 업데이트되었습니다.
2025-10-26 00:40:28,891 - modeling.models.svd - INFO - 🚀 SVD 추천 시스템 파이프라인 시작
2025-10-26 00:40:28,892 - modeling.models.svd - INFO - ============================================================
2025-10-26 00:40:28,952 - modeling.models.svd - INFO - === Surprise Dataset 준비 ===


[필터링 후] 사용자 수: 4,416명, 영화 수: 15,770개
필터링된 평점 수: 3,271,262개 (제거: 89,839개)


2025-10-26 00:40:29,739 - modeling.models.svd - INFO - ✅ 데이터셋 생성 완료
2025-10-26 00:40:29,739 - modeling.models.svd - INFO -   - 총 평점 수: 3,271,262
2025-10-26 00:40:29,740 - modeling.models.svd - INFO -   - 평점 범위: 0.5 ~ 5.0
2025-10-26 00:40:29,740 - modeling.models.svd - INFO - === Surprise Dataset 준비 ===
2025-10-26 00:40:29,740 - modeling.models.svd - INFO - ✅ 데이터셋 생성 완료
2025-10-26 00:40:29,741 - modeling.models.svd - INFO -   - 총 평점 수: 12
2025-10-26 00:40:29,741 - modeling.models.svd - INFO -   - 평점 범위: 0.5 ~ 5.0
2025-10-26 00:40:29,741 - modeling.models.svd - INFO - === Train/Test Split (test_size=0.2) ===
2025-10-26 00:40:32,201 - modeling.models.svd - INFO - === Surprise Dataset 준비 ===
2025-10-26 00:40:32,937 - modeling.models.svd - INFO - ✅ 데이터셋 생성 완료
2025-10-26 00:40:32,937 - modeling.models.svd - INFO -   - 총 평점 수: 3,271,274
2025-10-26 00:40:32,937 - modeling.models.svd - INFO -   - 평점 범위: 0.5 ~ 5.0
2025-10-26 00:40:33,886 - modeling.models.svd - INFO - ✅ 데이터 분할 완료
2025-10-26 00:4

...

2025-10-26 00:40:39,244 - modeling.models.svd - INFO - ✅ 학습 완료!
2025-10-26 00:40:39,245 - modeling.models.svd - INFO - === 모델 평가 ===
2025-10-26 00:40:39,245 - modeling.models.svd - INFO - Test set 평가:
2025-10-26 00:40:42,020 - modeling.models.svd - INFO - Train set 평가:


RMSE: 0.7084
MAE:  0.5371
RMSE: 0.7090


2025-10-26 00:40:50,963 - modeling.models.svd - INFO - 
=== 평가 결과 요약 ===
Train RMSE: 0.7090
Test RMSE:  0.7084
Train MAE:  0.5378
Test MAE:   0.5371

User Overlap: 100.00%
Item Overlap: 100.00%

2025-10-26 00:40:50,964 - modeling.models.svd - INFO - ✅ Train과 Test 성능이 비슷합니다. 적절한 일반화가 이루어졌습니다.


MAE:  0.5378


2025-10-26 00:40:51,253 - modeling.models.svd - INFO - ============================================================
2025-10-26 00:40:51,253 - modeling.models.svd - INFO - ✅ 파이프라인 완료!


In [3]:
from data_scraping.common.data_loader import load_movie_data

df_movies = load_movie_data()

In [3]:
if getattr(config, 'use_integrated_data', True) and 'firebase_data' in locals():
    print('hi')

hi


In [26]:
filtered_data['user_id'].values[-1] == 'SOMV3U3OEWT7LG9t2VMeNXKQchd2'

True

In [4]:
pipeline.recommend_for_user(user_id='SOMV3U3OEWT7LG9t2VMeNXKQchd2', df_movies=df_movies, n=10)

(Empty DataFrame
 Columns: [user_id, movie_id, movie_title, rating, title, year, genre, country, runtime, age_rating, cast, plot, avg_score, popularity, review_count]
 Index: [],
       movie_id  predicted_rating        title    year          genre  \
 5578   md6B3Ad          4.544781       대부 일대기  1992.0         드라마 범죄   
 15745  mJOVXR5          4.539241       다크 나이트  2008.0  액션 범죄 드라마 스릴러   
 2168   mNdKBkO          4.533777       쇼생크 탈출  1994.0            드라마   
 13461  mW4L2XW          4.519072          인셉션  2010.0   액션 모험 SF 스릴러   
 7239   m5r0pVO          4.480747  12인의 노한 사람들  1997.0         드라마 범죄   
 15535  mV539wO          4.463356     인생은 아름다워  1997.0    코미디 드라마 로맨스   
 9588   m1WwY4d          4.428120       시네마 천국  1988.0            드라마   
 7996   mdBzxzd          4.420564  12명의 성난 사람들  1957.0         범죄 드라마   
 13122  mO2p7x5          4.418770        인터스텔라  2014.0      모험 드라마 SF   
 4425   mLOPKgW          4.417624        트루먼 쇼  1998.0        코미디 드라마   
 
            coun